In [22]:
# %load degrees-of-separation.py
#Boilerplate stuff:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("DegreesOfSeparation")
sc = SparkContext(conf = conf)

# The characters we wish to find the degree of separation between:
startCharacterID = 888 #SpiderMan
targetCharacterID = 889  #ADAM 3,031 (who?)

# Our accumulator, used to signal when we find the target character during
# our BFS traversal.
hitCounter = sc.accumulator(0)

def convertToBFS(line):
    fields = line.split()
    heroID = int(fields[0])
    connections = []
    for connection in fields[1:]:
        connections.append(int(connection))

    color = 'WHITE'
    distance = 9999

    if (heroID == startCharacterID):
        color = 'GRAY'
        distance = 0

    return (heroID, (connections, distance, color))


def createStartingRdd():
    inputFile = sc.textFile("file:///Users/karthikramesh/SparkCourse/Data/marvel-graph.txt")
    return inputFile.map(convertToBFS)

def bfsMap(node):
    characterID = node[0]
    data = node[1]
    connections = data[0]
    distance = data[1]
    color = data[2]

    results = []

    #If this node needs to be expanded...
    if (color == 'GRAY'):
        for connection in connections:
            newCharacterID = connection
            newDistance = distance + 1
            newColor = 'GRAY'
            if (targetCharacterID == connection):
                hitCounter.add(1)

            newEntry = (newCharacterID, ([], newDistance, newColor))
            results.append(newEntry)
            print(results)

        #We've processed this node, so color it black
        color = 'BLACK'

    #Emit the input node so we don't lose it.
    results.append( (characterID, (connections, distance, color)) )
    return results

def bfsReduce(data1, data2):
    edges1 = data1[0]
    edges2 = data2[0]
    distance1 = data1[1]
    distance2 = data2[1]
    color1 = data1[2]
    color2 = data2[2]

    distance = 9999
    color = color1
    edges = []

    # See if one is the original node with its connections.
    # If so preserve them.
    if (len(edges1) > 0):
        edges.extend(edges1)
    if (len(edges2) > 0):
        edges.extend(edges2)

    # Preserve minimum distance
    if (distance1 < distance):
        distance = distance1

    if (distance2 < distance):
        distance = distance2

    # Preserve darkest color
    if (color1 == 'WHITE' and (color2 == 'GRAY' or color2 == 'BLACK')):
        color = color2

    if (color1 == 'GRAY' and color2 == 'BLACK'):
        color = color2

    if (color2 == 'WHITE' and (color1 == 'GRAY' or color1 == 'BLACK')):
        color = color1

    if (color2 == 'GRAY' and color1 == 'BLACK'):
        color = color1

    return (edges, distance, color)


#Main program here:
iterationRdd = createStartingRdd()

for iteration in range(0, 10):
    print("Running BFS iteration# " + str(iteration+1))

    # Create new vertices as needed to darken or reduce distances in the
    # reduce stage. If we encounter the node we're looking for as a GRAY
    # node, increment our accumulator to signal that we're done.
    mapped = iterationRdd.flatMap(bfsMap)

    # Note that mapped.count() action here forces the RDD to be evaluated, and
    # that's the only reason our accumulator is actually updated.
    print("Processing " + str(mapped.count()) + " values.")

    if (hitCounter.value > 0):
        print("Hit the target character! From " + str(hitCounter.value) \
            + " different direction(s).")
        break

    # Reducer combines data for each character ID, preserving the darkest
    # color and shortest path.
    iterationRdd = mapped.reduceByKey(bfsReduce)


Running BFS iteration# 1
Processing 6596 values.
Running BFS iteration# 2
Processing 7567 values.
Hit the target character! From 1 different direction(s).


In [21]:
sc.stop()

In [34]:
# Testing


# The characters we wish to find the degree of separation between:
startCharacterID = 888 #SpiderMan
targetCharacterID = 889  #ADAM 3,031 (who?)

# Our accumulator, used to signal when we find the target character during
# our BFS traversal.
hitCounter = sc.accumulator(0)

def convertToBFS(line):
    fields = line.split()
    heroID = int(fields[0])
    connections = []
    for connection in fields[1:]:
        connections.append(int(connection))

    color = 'WHITE'
    distance = 9999

    if (heroID == startCharacterID):
        color = 'GRAY'
        distance = 0

    return (heroID, (connections, distance, color))


def createStartingRdd():
    inputFile = sc.textFile("file:///Users/karthikramesh/SparkCourse/Data/marvel-graph.txt")
    return inputFile.map(convertToBFS)

bfs_data = createStartingRdd()

a = bfs_data.take(1)
a[0]


(5988,
 ([748,
   1722,
   3752,
   4655,
   5743,
   1872,
   3413,
   5527,
   6368,
   6085,
   4319,
   4728,
   1636,
   2397,
   3364,
   4001,
   1614,
   1819,
   1585,
   732,
   2660,
   3952,
   2507,
   3891,
   2070,
   2239,
   2602,
   612,
   1352,
   5447,
   4548,
   1596,
   5488,
   1605,
   5517,
   11,
   479,
   2554,
   2043,
   17,
   865,
   4292,
   6312,
   473,
   534,
   1479,
   6375,
   4456],
  9999,
  'WHITE'))

In [35]:
def bfsMap(node):
    characterID = node[0]
    data = node[1]
    connections = data[0]
    distance = data[1]
    color = data[2]

    results = []

    #If this node needs to be expanded...
    if (color == 'WHITE'):
        for connection in connections:
            newCharacterID = connection
            newDistance = distance + 1
            newColor = 'GRAY'
            print(newCharacterID)
            print(newDistance)
            print(newColor)
            if (targetCharacterID == connection):
                hitCounter.add(1)

            newEntry = (newCharacterID, ([], newDistance, newColor))
            results.append(newEntry)

        #We've processed this node, so color it black
        color = 'BLACK'

    #Emit the input node so we don't lose it.
    results.append( (characterID, (connections, distance, color)) )
    print(results)
    return results

result = bfsMap(a[0])

748
10000
GRAY
1722
10000
GRAY
3752
10000
GRAY
4655
10000
GRAY
5743
10000
GRAY
1872
10000
GRAY
3413
10000
GRAY
5527
10000
GRAY
6368
10000
GRAY
6085
10000
GRAY
4319
10000
GRAY
4728
10000
GRAY
1636
10000
GRAY
2397
10000
GRAY
3364
10000
GRAY
4001
10000
GRAY
1614
10000
GRAY
1819
10000
GRAY
1585
10000
GRAY
732
10000
GRAY
2660
10000
GRAY
3952
10000
GRAY
2507
10000
GRAY
3891
10000
GRAY
2070
10000
GRAY
2239
10000
GRAY
2602
10000
GRAY
612
10000
GRAY
1352
10000
GRAY
5447
10000
GRAY
4548
10000
GRAY
1596
10000
GRAY
5488
10000
GRAY
1605
10000
GRAY
5517
10000
GRAY
11
10000
GRAY
479
10000
GRAY
2554
10000
GRAY
2043
10000
GRAY
17
10000
GRAY
865
10000
GRAY
4292
10000
GRAY
6312
10000
GRAY
473
10000
GRAY
534
10000
GRAY
1479
10000
GRAY
6375
10000
GRAY
4456
10000
GRAY
[(748, ([], 10000, 'GRAY')), (1722, ([], 10000, 'GRAY')), (3752, ([], 10000, 'GRAY')), (4655, ([], 10000, 'GRAY')), (5743, ([], 10000, 'GRAY')), (1872, ([], 10000, 'GRAY')), (3413, ([], 10000, 'GRAY')), (5527, ([], 10000, 'GRAY')), (6368, ([],

In [30]:
result

[(5988,
  ([748,
    1722,
    3752,
    4655,
    5743,
    1872,
    3413,
    5527,
    6368,
    6085,
    4319,
    4728,
    1636,
    2397,
    3364,
    4001,
    1614,
    1819,
    1585,
    732,
    2660,
    3952,
    2507,
    3891,
    2070,
    2239,
    2602,
    612,
    1352,
    5447,
    4548,
    1596,
    5488,
    1605,
    5517,
    11,
    479,
    2554,
    2043,
    17,
    865,
    4292,
    6312,
    473,
    534,
    1479,
    6375,
    4456],
   9999,
   'WHITE'))]